Parameter searching: https://www.projectpro.io/recipes/find-optimal-parameters-using-gridsearchcv \
Renaming the last column: https://stackoverflow.com/questions/56479835/rename-only-the-last-column-in-pandas-dataframe-accounting-for-duplicate-header

Preperation

In [1]:
import pandas
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC

c:\Users\victo\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
import sklearn
print(sklearn.__version__)

1.0.2


In [2]:
#loading in dataset normal, STM and BerTopic
df = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Sample\\sample.csv")
df_STM = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Sample\\features_sample_STM.csv")
df_Bert = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Sample\\features_sample_Bert_reduced.csv")
df_STM_selected = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Sample\\Sample_featured_selected_STM.csv")
df_Bert_selected = pandas.read_csv("G:\\Master\\Block 3\\Thesis\\Sample\\Sample_featured_selected_Bert.csv")

In [4]:
df_STM.columns = [*df_STM.columns[:-1], 'sentiment']

In [5]:
df_STM

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V112,V113,V114,V115,V116,V117,V118,V119,V120,sentiment
0,0.001030,0.002027,0.000527,0.000378,0.001124,0.039118,0.012707,0.006189,0.003038,0.001320,...,0.004531,0.002872,0.001559,0.000962,0.014338,0.001158,0.008602,0.010092,0.002836,0
1,0.000851,0.001892,0.000647,0.000380,0.001040,0.029230,0.006719,0.052227,0.007085,0.001319,...,0.014664,0.004294,0.001827,0.000721,0.002362,0.001433,0.004187,0.007426,0.003151,1
2,0.000708,0.001753,0.000584,0.000282,0.000680,0.030866,0.010538,0.006753,0.017031,0.001409,...,0.210494,0.019108,0.001475,0.000930,0.001968,0.000651,0.005079,0.021736,0.002512,0
3,0.001218,0.005576,0.012048,0.008318,0.001654,0.045555,0.005865,0.007628,0.003562,0.001704,...,0.020091,0.002962,0.000892,0.001102,0.017580,0.001325,0.008031,0.054394,0.003069,0
4,0.001521,0.003697,0.000844,0.000252,0.001784,0.039413,0.005086,0.021152,0.013926,0.000902,...,0.009288,0.000816,0.002093,0.000774,0.001825,0.002602,0.002455,0.005683,0.002809,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,0.001353,0.108549,0.000977,0.000345,0.003568,0.057887,0.007655,0.040965,0.016742,0.000837,...,0.003300,0.001532,0.001557,0.002331,0.002155,0.004349,0.002952,0.063389,0.003008,0
25885,0.001054,0.007069,0.074855,0.000221,0.001314,0.032114,0.008504,0.043870,0.004286,0.000655,...,0.001840,0.000963,0.000830,0.032408,0.046063,0.001052,0.005064,0.004210,0.002734,1
25886,0.000753,0.001885,0.001025,0.000213,0.001507,0.019102,0.005220,0.021911,0.061482,0.000588,...,0.012958,0.000560,0.001571,0.000825,0.001197,0.002342,0.001518,0.005652,0.002274,1
25887,0.002256,0.002623,0.000846,0.000277,0.002363,0.045607,0.017417,0.012401,0.004876,0.000957,...,0.022726,0.001375,0.002251,0.013894,0.025253,0.000847,0.002625,0.002722,0.002950,0


In [6]:
df_Bert

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,111,112,113,114,115,116,117,118,119,sentiment
0,0,2.406603e-02,1.631666e-01,2.190369e-02,6.570173e-03,2.307655e-02,8.365412e-03,1.044651e-03,1.436761e-02,7.665692e-03,...,1.605185e-03,8.505731e-03,4.350101e-04,1.077927e-03,1.680891e-03,1.544089e-03,1.067855e-03,1.870928e-03,1.215840e-03,0
1,1,4.937321e-306,9.402316e-306,5.238678e-306,2.618513e-306,2.077104e-305,1.000000e+00,6.243770e-307,6.344945e-306,1.919045e-306,...,3.224686e-307,3.406447e-307,2.460309e-307,5.507471e-307,1.134362e-306,4.168658e-306,1.944866e-307,5.278984e-307,1.474514e-306,1
2,2,1.609316e-02,3.192536e-02,2.046305e-02,1.055913e-02,5.270181e-02,8.974301e-02,2.049910e-03,1.965568e-02,6.258281e-03,...,1.221447e-03,1.183277e-03,7.689357e-04,1.690983e-03,5.492027e-03,6.932800e-03,5.895996e-04,1.672702e-03,3.622286e-03,0
3,3,2.426547e-02,1.562201e-01,2.260713e-02,6.744916e-03,2.336913e-02,8.507393e-03,1.069602e-03,1.459541e-02,7.799090e-03,...,1.664065e-03,9.642235e-03,4.452774e-04,1.096405e-03,1.713376e-03,1.565868e-03,1.098450e-03,1.889294e-03,1.235315e-03,0
4,4,6.190984e-306,1.572890e-305,7.223648e-306,3.001427e-306,1.666965e-305,8.542622e-306,5.309188e-307,6.200085e-306,2.160959e-306,...,4.320584e-307,4.865545e-307,2.079111e-307,4.988167e-307,1.660531e-306,1.586303e-306,2.187228e-307,6.287674e-307,8.428811e-307,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25896,25896,1.320379e-02,3.576054e-02,3.501184e-02,1.040078e-02,2.400560e-02,1.242579e-02,1.390931e-03,1.207024e-02,4.813511e-03,...,1.777248e-03,1.608226e-03,5.245896e-04,9.712274e-04,4.733079e-03,1.968536e-03,5.950869e-04,1.239214e-03,1.355423e-03,0
25897,25897,4.529771e-03,1.179237e-02,7.929149e-03,2.164511e-03,5.079875e-03,2.026990e-03,4.415183e-04,3.359419e-03,1.820484e-03,...,7.053673e-04,1.414617e-03,1.840088e-04,2.616364e-04,4.209857e-04,3.494251e-04,4.217197e-04,3.731257e-04,3.070442e-04,1
25898,25898,1.553569e-02,3.046587e-02,2.226832e-02,1.255153e-02,4.626328e-02,2.650033e-01,2.475263e-03,1.914384e-02,6.250478e-03,...,1.313843e-03,1.202768e-03,8.912865e-04,1.679637e-03,6.112104e-03,5.255948e-03,6.175675e-04,1.593530e-03,3.395612e-03,1
25899,25899,4.562475e-03,1.180991e-02,7.923741e-03,2.173168e-03,5.106683e-03,2.035990e-03,4.444279e-04,3.383648e-03,1.836676e-03,...,7.050692e-04,1.407580e-03,1.853717e-04,2.634941e-04,4.219986e-04,3.510347e-04,4.273203e-04,3.756459e-04,3.087271e-04,0


In [3]:
df_STM_selected

,V1,V2,V3,V4,V6,V7,V8,V9,V10,V11,...,V111,V112,V113,V114,V116,V117,V118,V119,V120,sentiment
0,0.001030,0.002027,0.000527,0.000378,0.039118,0.012707,0.006189,0.003038,0.001320,0.003013,...,0.002468,0.004531,0.002872,0.001559,0.014338,0.001158,0.008602,0.010092,0.002836,0
1,0.000851,0.001892,0.000647,0.000380,0.029230,0.006719,0.052227,0.007085,0.001319,0.001721,...,0.012356,0.014664,0.004294,0.001827,0.002362,0.001433,0.004187,0.007426,0.003151,1
2,0.000708,0.001753,0.000584,0.000282,0.030866,0.010538,0.006753,0.017031,0.001409,0.001778,...,0.003094,0.210494,0.019108,0.001475,0.001968,0.000651,0.005079,0.021736,0.002512,0
3,0.001218,0.005576,0.012048,0.008318,0.045555,0.005865,0.007628,0.003562,0.001704,0.001617,...,0.002371,0.020091,0.002962,0.000892,0.017580,0.001325,0.008031,0.054394,0.003069,0
4,0.001521,0.003697,0.000844,0.000252,0.039413,0.005086,0.021152,0.013926,0.000902,0.004319,...,0.003226,0.009288,0.000816,0.002093,0.001825,0.002602,0.002455,0.005683,0.002809,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25884,0.001353,0.108549,0.000977,0.000345,0.057887,0.007655,0.040965,0.016742,0.000837,0.002989,...,0.014499,0.003300,0.001532,0.001557,0.002155,0.004349,0.002952,0.063389,0.003008,0
25885,0.001054,0.007069,0.074855,0.000221,0.032114,0.008504,0.043870,0.004286,0.000655,0.003588,...,0.002434,0.001840,0.000963,0.000830,0.046063,0.001052,0.005064,0.004210,0.002734,1
25886,0.000753,0.001885,0.001025,0.000213,0.019102,0.005220,0.021911,0.061482,0.000588,0.024373,...,0.008500,0.012958,0.000560,0.001571,0.001197,0.002342,0.001518,0.005652,0.002274,1
25887,0.002256,0.002623,0.000846,0.000277,0.045607,0.017417,0.012401,0.004876,0.000957,0.001413,...,0.001832,0.022726,0.001375,0.002251,0.025253,0.000847,0.002625,0.002722,0.002950,0


In [4]:
df_Bert_selected

,0,1,4,5,6,7,8,9,11,12,...,108,109,111,112,115,116,117,118,119,sentiment
0,2.406603e-02,1.631666e-01,2.307655e-02,8.365412e-03,1.044651e-03,1.436761e-02,7.665692e-03,1.521247e-02,2.381878e-02,4.134728e-02,...,7.608225e-04,2.045898e-03,1.605185e-03,8.505731e-03,1.680891e-03,1.544089e-03,1.067855e-03,1.870928e-03,1.215840e-03,0
1,4.937321e-306,9.402316e-306,2.077104e-305,1.000000e+00,6.243770e-307,6.344945e-306,1.919045e-306,9.030316e-306,1.043084e-305,8.912841e-306,...,3.389193e-307,9.537043e-307,3.224686e-307,3.406447e-307,1.134362e-306,4.168658e-306,1.944866e-307,5.278984e-307,1.474514e-306,1
2,1.609316e-02,3.192536e-02,5.270181e-02,8.974301e-02,2.049910e-03,1.965568e-02,6.258281e-03,2.901898e-02,4.356667e-02,3.244086e-02,...,1.021346e-03,2.771639e-03,1.221447e-03,1.183277e-03,5.492027e-03,6.932800e-03,5.895996e-04,1.672702e-03,3.622286e-03,0
3,2.426547e-02,1.562201e-01,2.336913e-02,8.507393e-03,1.069602e-03,1.459541e-02,7.799090e-03,1.532603e-02,2.427684e-02,4.111136e-02,...,7.750611e-04,2.060542e-03,1.664065e-03,9.642235e-03,1.713376e-03,1.565868e-03,1.098450e-03,1.889294e-03,1.235315e-03,0
4,6.190984e-306,1.572890e-305,1.666965e-305,8.542622e-306,5.309188e-307,6.200085e-306,2.160959e-306,1.000000e+00,2.182230e-305,5.088318e-305,...,3.103458e-307,1.152121e-306,4.320584e-307,4.865545e-307,1.660531e-306,1.586303e-306,2.187228e-307,6.287674e-307,8.428811e-307,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25896,1.320379e-02,3.576054e-02,2.400560e-02,1.242579e-02,1.390931e-03,1.207024e-02,4.813511e-03,1.744991e-02,2.354585e-01,3.799453e-02,...,6.577460e-04,1.761583e-03,1.777248e-03,1.608226e-03,4.733079e-03,1.968536e-03,5.950869e-04,1.239214e-03,1.355423e-03,0
25897,4.529771e-03,1.179237e-02,5.079875e-03,2.026990e-03,4.415183e-04,3.359419e-03,1.820484e-03,2.962209e-03,5.951839e-03,5.865877e-03,...,2.344190e-04,3.964906e-04,7.053673e-04,1.414617e-03,4.209857e-04,3.494251e-04,4.217197e-04,3.731257e-04,3.070442e-04,1
25898,1.553569e-02,3.046587e-02,4.626328e-02,2.650033e-01,2.475263e-03,1.914384e-02,6.250478e-03,2.405524e-02,4.629650e-02,2.907926e-02,...,1.043547e-03,2.478488e-03,1.313843e-03,1.202768e-03,6.112104e-03,5.255948e-03,6.175675e-04,1.593530e-03,3.395612e-03,1
25899,4.562475e-03,1.180991e-02,5.106683e-03,2.035990e-03,4.444279e-04,3.383648e-03,1.836676e-03,2.973211e-03,5.955003e-03,5.867759e-03,...,2.362549e-04,3.984161e-04,7.050692e-04,1.407580e-03,4.219986e-04,3.510347e-04,4.273203e-04,3.756459e-04,3.087271e-04,0


In [7]:
#vectorizing the bag of words
vectorizer_model = CountVectorizer(min_df = 5)


In [8]:
#splitting the columns in X and y
bow_X = vectorizer_model.fit_transform(df["review_text"].values)
bow_y = df["review_score"].values
STM_X = df_STM.drop("sentiment", axis=1)
STM_X = STM_X.values
STM_y = df_STM["sentiment"].values
STM_X_selected = df_STM_selected.drop("sentiment", axis = 1)
STM_y_selected = df_STM_selected["sentiment"].values
Bert_X = df_Bert.drop(["sentiment", "Unnamed: 0"], axis=1)
Bert_X = Bert_X.values
Bert_y = df_Bert["sentiment"].values
Bert_X_selected = df_Bert_selected.drop("sentiment", axis = 1)
Bert_y_selected = df_Bert_selected["sentiment"].values

In [9]:
bow_X

<25890x16124 sparse matrix of type '<class 'numpy.int64'>'
	with 1985300 stored elements in Compressed Sparse Row format>

In [10]:
#splitting the sets
bow_X_train, bow_X_test, bow_y_train, bow_y_test = train_test_split(bow_X, bow_y, random_state = 101)
STM_X_train, STM_X_test, STM_y_train, STM_y_test = train_test_split(STM_X, STM_y, random_state = 101)
Bert_X_train, Bert_X_test, Bert_y_train, Bert_y_test = train_test_split(Bert_X, Bert_y, random_state = 101)
STM_X_selected_train, STM_X_selected_test, STM_y_selected_train, STM_y_selected_test = train_test_split(STM_X_selected,STM_y_selected, random_state=101)
Bert_X_selected_train, Bert_X_selected_test, Bert_y_selected_train, Bert_y_selected_test = train_test_split(Bert_X_selected,Bert_y_selected, random_state=101)

In [11]:
le_bow = preprocessing.LabelEncoder()
bow_y_train = le_bow.fit_transform(bow_y_train)
bow_y_test = le_bow.transform(bow_y_test)

In [12]:
le_stm = preprocessing.LabelEncoder()
STM_y_train = le_stm.fit_transform(STM_y_train)
STM_y_test = le_stm.transform(STM_y_test)

In [13]:
le_bert = preprocessing.LabelEncoder()
Bert_y_train = le_bert.fit_transform(Bert_y_train)
Bert_y_test = le_bert.transform(Bert_y_test)

In [14]:
le_stm_selected = preprocessing.LabelEncoder()
STM_y_selected_train = le_stm.fit_transform(STM_y_selected_train)
STM_y_selected_test = le_stm.transform(STM_y_selected_test)

In [15]:
le_bert_selected = preprocessing.LabelEncoder()
Bert_y_selected_train = le_bert.fit_transform(Bert_y_selected_train)
Bert_y_selected_test = le_bert.transform(Bert_y_selected_test)

Logistic Regression

In [16]:
LR = LogisticRegression(random_state=101, max_iter=1000) 

In [17]:
parameters_LR = {"C": np.logspace(-4, 4, 20), "class_weight":[None, "balanced"], "solver": ["liblinear", "lbfgs"]}

In [18]:
#BOW
Randomized_search_LR_BOW = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_LR_BOW.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_LR_BOW.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_LR_BOW.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_LR_BOW.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LogisticRegression(C=0.004832930238571752, class_weight='balanced',
                   max_iter=1000, random_state=101)
\\n The best score across ALL searched params:\\n 0.732832333973113
\\n The best parameters across ALL searched params:\\n {'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 0.004832930238571752}


In [19]:
#BOW final
LR_final_BOW = Randomized_search_LR_BOW.best_estimator_
LR_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_LR = LR_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_LR)

0.740098429810171

In [20]:
#STM
Randomized_search_LR_STM = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_LR_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_LR_STM.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_LR_STM.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_LR_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LogisticRegression(C=206.913808111479, class_weight='balanced', max_iter=1000,
                   random_state=101)
\\n The best score across ALL searched params:\\n 0.6935608799722481
\\n The best parameters across ALL searched params:\\n {'solver': 'lbfgs', 'class_weight': 'balanced', 'C': 206.913808111479}


In [21]:
#STM final
LR_final_STM = Randomized_search_LR_STM.best_estimator_
LR_final_STM.fit(STM_X_train, STM_y_train)
STM_y_pred_LR = LR_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, STM_y_pred_LR)

0.6927320700407771

In [22]:
#STM selected
Randomized_search_LR_STM_selected = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_LR_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_LR_STM_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_LR_STM_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_LR_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LogisticRegression(C=206.913808111479, class_weight='balanced', max_iter=1000,
                   random_state=101, solver='liblinear')
\\n The best score across ALL searched params:\\n 0.6920081337915035
\\n The best parameters across ALL searched params:\\n {'solver': 'liblinear', 'class_weight': 'balanced', 'C': 206.913808111479}


In [23]:
#STM selected final
LR_final_STM_selected = Randomized_search_LR_STM_selected.best_estimator_
LR_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_LR = LR_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_LR)

0.6910490856592877

In [24]:
#BERTopic
Randomized_search_LR_Bert = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_LR_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_LR_Bert.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_LR_Bert.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_LR_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LogisticRegression(C=0.004832930238571752, class_weight='balanced',
                   max_iter=1000, random_state=101, solver='liblinear')
\\n The best score across ALL searched params:\\n 0.4330325158280076
\\n The best parameters across ALL searched params:\\n {'solver': 'liblinear', 'class_weight': 'balanced', 'C': 0.004832930238571752}


In [25]:
#BERT final 0.48 veel features, 0.42 outlier reduction
LR_final_Bert = Randomized_search_LR_Bert.best_estimator_
LR_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_LR = LR_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_LR)

0.4268719384184745

In [26]:
#Bertopic selected
Randomized_search_LR_Bert_selected = RandomizedSearchCV(LR, parameters_LR, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_LR_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_LR_Bert_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_LR_Bert_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_LR_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LogisticRegression(C=0.0018329807108324356, class_weight='balanced',
                   max_iter=1000, random_state=101, solver='liblinear')
\\n The best score across ALL searched params:\\n 0.4301687813309484
\\n The best parameters across ALL searched params:\\n {'solver': 'liblinear', 'class_weight': 'balanced', 'C': 0.0018329807108324356}


In [27]:
#Bertopic selected final
LR_final_Bert_selected = Randomized_search_LR_Bert_selected.best_estimator_
LR_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_LR = LR_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_LR)

0.42118047121690555

Linear Support vector Machine

In [28]:
SVM = LinearSVC(random_state=101)

In [29]:
parameters_SVM = {"C": np.logspace(-4, 4, 20), "class_weight":[None, "balanced"], "loss": ["hinge", "squared_hinge"]} 

In [30]:
#BOW
Randomized_search_SVM_BOW = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_SVM_BOW.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_SVM_BOW.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_SVM_BOW.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_SVM_BOW.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LinearSVC(C=0.08858667904100823, class_weight='balanced', loss='hinge',
          random_state=101)
\\n The best score across ALL searched params:\\n 0.7111118894778182
\\n The best parameters across ALL searched params:\\n {'loss': 'hinge', 'class_weight': 'balanced', 'C': 0.08858667904100823}


c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [31]:
#BOW final
SVM_final_BOW = Randomized_search_SVM_BOW.best_estimator_
SVM_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_SVM = SVM_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_SVM)

c:\Users\victo\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7160375090165905

In [32]:
#STM
Randomized_search_SVM_STM = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_SVM_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_SVM_STM.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_SVM_STM.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_SVM_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LinearSVC(C=4.281332398719396, class_weight='balanced', random_state=101)
\\n The best score across ALL searched params:\\n 0.6914690436096311
\\n The best parameters across ALL searched params:\\n {'loss': 'squared_hinge', 'class_weight': 'balanced', 'C': 4.281332398719396}


In [33]:
#STM final
SVM_final_STM = Randomized_search_SVM_STM.best_estimator_
SVM_final_STM.fit(STM_X_train, STM_y_train)
STM_y_pred_SVM = SVM_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, STM_y_pred_SVM)

0.6893203883495146

In [34]:
#STM selected
Randomized_search_SVM_STM_selected = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_SVM_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_SVM_STM_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_SVM_STM_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_SVM_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LinearSVC(C=1.623776739188721, class_weight='balanced', random_state=101)
\\n The best score across ALL searched params:\\n 0.6897605376201437
\\n The best parameters across ALL searched params:\\n {'loss': 'squared_hinge', 'class_weight': 'balanced', 'C': 1.623776739188721}


In [35]:
#STM selected final
SVM_final_STM_selected = Randomized_search_SVM_STM_selected.best_estimator_
SVM_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_SVM = SVM_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_SVM)

0.6882295719844359

In [36]:
#BERTopic
Randomized_search_SVM_Bert = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_SVM_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_SVM_Bert.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_SVM_Bert.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_SVM_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LinearSVC(C=0.0001, class_weight='balanced', random_state=101)
\\n The best score across ALL searched params:\\n 0.436066395322654
\\n The best parameters across ALL searched params:\\n {'loss': 'squared_hinge', 'class_weight': 'balanced', 'C': 0.0001}


In [37]:
#BERT final 0.47, 0.42 reduction
SVM_final_Bert = Randomized_search_SVM_Bert.best_estimator_
SVM_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_SVM = SVM_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_SVM)

0.42896299680219274

In [38]:
#Bertopic selected
Randomized_search_SVM_Bert_selected = RandomizedSearchCV(SVM, parameters_SVM, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_SVM_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_SVM_Bert_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_SVM_Bert_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_SVM_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n LinearSVC(C=0.0018329807108324356, class_weight='balanced', loss='hinge',
          random_state=101)
\\n The best score across ALL searched params:\\n 0.45925395981177325
\\n The best parameters across ALL searched params:\\n {'loss': 'hinge', 'class_weight': 'balanced', 'C': 0.0018329807108324356}


In [39]:
#Bertopic selected final
SVM_final_Bert_selected = Randomized_search_SVM_Bert_selected.best_estimator_
SVM_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_SVM = SVM_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_SVM)

0.44657332350773765

XGBClassifier

In [40]:
XGB = XGBClassifier(verbosity = 1, seed = 101, use_label_encoder=False)

In [41]:

parameters_XGB = {"colsample_bytree:": np.arange(0.5,1,0.1) ,"min_child_weight": np.arange(1,10,1), "eta": np.arange(0.01,0.3,0.05), "gamma": np.arange(0,5,1), "max_depth": np.arange(3,10,1), "subsample": np.arange(0.5,1,0.1), "scale_pos_weight": [1, 4.045069258], "objective": ["binary:logistic", "binary:logitraw", "binary:hinge"]}

In [42]:
#BOW
Randomized_search_XGB = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_XGB.fit(bow_X_train, bow_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_XGB.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_XGB.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_XGB.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[16:35:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:35:15] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              c

In [43]:
#BOW final
XGB_final_BOW = Randomized_search_XGB.best_estimator_
XGB_final_BOW.fit(bow_X_train, bow_y_train)
bow_y_pred_XGB = XGB_final_BOW.predict(bow_X_test)
#test score
f1_score(bow_y_test, bow_y_pred_XGB)

[16:35:17] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:35:17] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6908167974157822

In [44]:
#STM
Randomized_search_XGB_STM = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_XGB_STM.fit(STM_X_train, STM_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_XGB_STM.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_XGB_STM.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_XGB_STM.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[16:41:58] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:41:58] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              c

In [45]:
#STM final
XGB_final_STM = Randomized_search_XGB_STM.best_estimator_
XGB_final_STM.fit(STM_X_train, STM_y_train)
XGB_y_pred_STM = XGB_final_STM.predict(STM_X_test)
#test score
f1_score(STM_y_test, XGB_y_pred_STM)

[16:42:07] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:42:07] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7020621000237023

In [46]:
#STM selected
Randomized_search_XGB_STM_selected = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_XGB_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_XGB_STM_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_XGB_STM_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_XGB_STM_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\victo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[16:49:23] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:49:23] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, colsample_bytree:=0.

In [47]:
#STM selected final
XGB_final_STM_selected = Randomized_search_XGB_STM_selected.best_estimator_
XGB_final_STM_selected.fit(STM_X_selected_train, STM_y_selected_train)
STM_y_selected_pred_XGB = XGB_final_STM_selected.predict(STM_X_selected_test)
#test score
f1_score(STM_y_selected_test, STM_y_selected_pred_XGB)

[16:49:44] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:49:44] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.7057449789817841

In [48]:
#BERTopic
Randomized_search_XGB_Bert = RandomizedSearchCV(XGB, parameters_XGB, verbose=2, scoring="f1", n_jobs = 2)
Randomized_search_XGB_Bert.fit(Bert_X_train, Bert_y_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_XGB_Bert.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_XGB_Bert.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_XGB_Bert.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[16:55:12] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              colsample_bytree:=0.7999999999999999, enable_categorical=False,
              eta=0.01, gamma=4, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.00999999978,
              max_delta_step=0, max_depth=7, min_ch

In [49]:
#BERT final
XGB_final_Bert = Randomized_search_XGB_Bert.best_estimator_
XGB_final_Bert.fit(Bert_X_train, Bert_y_train)
Bert_y_pred_XGB = XGB_final_Bert.predict(Bert_X_test)
#test score
f1_score(Bert_y_test, Bert_y_pred_XGB)

[16:55:27] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:", "scale_pos_weight" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.47160214749929363

In [50]:
#Bertopic selected
Randomized_search_XGB_Bert_selected = RandomizedSearchCV(XGB, parameters_XGB, verbose=1, scoring="f1", n_jobs = 2)
Randomized_search_XGB_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
print(" Results from Randomized Search " )
print("\\\\n The best estimator across ALL searched params:\\\\n",Randomized_search_XGB_Bert_selected.best_estimator_)
print("\\\\n The best score across ALL searched params:\\\\n",Randomized_search_XGB_Bert_selected.best_score_)
print("\\\\n The best parameters across ALL searched params:\\\\n",Randomized_search_XGB_Bert_selected.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\victo\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[16:59:00] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:59:00] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 Results from Randomized Search 
\\n The best estimator across ALL searched params:\\n XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1,
              colsam

In [51]:
#Bertopic selected final
XGB_final_Bert_selected = Randomized_search_XGB_Bert_selected.best_estimator_
XGB_final_Bert_selected.fit(Bert_X_selected_train, Bert_y_selected_train)
Bert_y_selected_pred_XGB = XGB_final_Bert_selected.predict(Bert_X_selected_test)
#test score
f1_score(Bert_y_selected_test, Bert_y_selected_pred_XGB)

[16:59:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:576: 
Parameters: { "colsample_bytree:" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:59:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.4732018883643433